In [1]:
# -*- coding: utf-8 -*-
import os
import pandas as pd
import numpy as np
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import gc, math
import pickle

from keras.models import Sequential
from keras.optimizers import SGD
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.models import Model, load_model, model_from_json

from keras.layers import Input, Dense, Convolution2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D, Dropout, Flatten, merge, Reshape, Activation
from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras.layers.normalization import BatchNormalization
from keras import regularizers
from keras import backend as K
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard

from sklearn.metrics import log_loss, accuracy_score, confusion_matrix

from cnnmodels import vgg_std16_model, preprocess_input, create_rect5, load_img, train_generator, test_generator
from cnnmodels import identity_block, testcv_generator, conv_block, resnet50_model
from cnnmodels import conv2d_bn, block_inception_a, block_reduction_a, block_inception_b, block_reduction_b
from cnnmodels import block_inception_c, inception_v4_base, inception_v4_model

Using Theano backend.
Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [2]:
# Params
img_rows, img_cols = 299, 299 # Resolution of inputs
channel = 3
ROWS, COLS = 299, 299
CHECKPOINT_DIR = 'log/checkpoint07/'
BATCHSIZE = 32
CERV_CLASSES = ['Type_1', 'Type_2', 'Type_3']
nb_perClass = int(BATCHSIZE / len(CERV_CLASSES))
TRAIN_DIR = '../data/original/train'
TEST_DIR = '../data/original/test'
DATA_DIR = '../data/original'
num_class = len(CERV_CLASSES)
full = False
bags = 5

In [3]:
train_datagen = ImageDataGenerator(
    rotation_range=180,
    shear_range=0.2,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=True)

In [4]:
img_ls = []
y_ls = []
for typ in CERV_CLASSES:
    for img in os.listdir(os.path.join(TRAIN_DIR, typ)):
        if img != '.DS_Store':
            img_ls.append(os.path.join(TRAIN_DIR, typ, img))
            y_ls.append(typ)
for typ in CERV_CLASSES:
    for img in os.listdir(os.path.join(DATA_DIR, typ)):
        if img != '.DS_Store':
            img_ls.append(os.path.join(DATA_DIR, typ, img))
            y_ls.append(typ)
train_all  = pd.DataFrame({'class': y_ls, 'img': img_ls, })[['img', 'class']]

In [5]:
img_ls = []
for img in os.listdir(TEST_DIR):
    if img != '.DS_Store':
        img_ls.append(os.path.join(TEST_DIR, img))
test_df  = pd.DataFrame({'img': img_ls}) 

In [6]:
train_all.tail(3)

,img,class
8206,../data/original/Type_3/5391.jpg,Type_3
8207,../data/original/Type_3/4116.jpg,Type_3
8208,../data/original/Type_3/568.jpg,Type_3


In [7]:
def train_generator(datagen, df):
    while 1:
        batch_x = np.zeros((BATCHSIZE, ROWS, COLS, 3), dtype=K.floatx())
        batch_y = np.zeros((BATCHSIZE, len(CERV_CLASSES)), dtype=K.floatx())
        fn = lambda obj: obj.loc[np.random.choice(obj.index, size=nb_perClass, replace=False),:]
        batch_df = df.groupby('class', as_index=True).apply(fn)
        i = 0
        for index,row in batch_df.iterrows():
            row = row.tolist()
            image_file = row[0]
            typ_class = row[1]
            img = Image.open(image_file).resize((ROWS, COLS))
            img = img.convert('RGB')
            x = np.asarray(img, dtype=K.floatx())
            #x = datagen.random_transform(x)
            x = preprocess_input(x)
            batch_x[i] = x
            batch_y[i,CERV_CLASSES.index(typ_class)] = 1
            i += 1
        #return (batch_x, batch_y)
        yield (batch_x.transpose(0, 3, 1, 2), batch_y)

In [8]:
# Split into train and valid
valid_set = pd.read_csv("../val_images.csv", header = None, names = ['img']).img.tolist()
valid_set[-4:]

['Type_2/3498.jpg', 'Type_2/1341.jpg', 'Type_3/6017.jpg', 'Type_2/5629.jpg']

In [9]:
valid_df = train_all[train_all['img'].str.replace('../data/original/', '').isin(valid_set)]
if full == True:
    train_df = train_all
else:
    train_df = train_all[~train_all['img'].str.replace('../data/original/', '').isin(valid_set)]
samples_per_epoch=BATCHSIZE*math.ceil(train_df.groupby('class').size()['Type_2']/nb_perClass)
print(train_df.shape)
print(valid_df.shape)

(8209, 2)
(1641, 2)


In [10]:
#samples_per_epoch = 5000

In [11]:
# Make our validation set
l = valid_df.groupby('class').size()
valid_x = np.zeros((valid_df.shape[0], ROWS, COLS, 3), dtype=K.floatx())
valid_y = np.zeros((valid_df.shape[0], len(CERV_CLASSES)), dtype=K.floatx())
i = 0
for index,row in valid_df.iterrows():
    row = row.tolist()
    image_file = row[0]
    typ_class = row[1]
    img = Image.open(image_file).resize((ROWS, COLS))
    img = img.convert('RGB')
    x = np.asarray(img, dtype=K.floatx())
    # x = datagen.random_transform(x)
    x = preprocess_input(x)
    valid_x[i] = x
    valid_y[i,CERV_CLASSES.index(typ_class)] = 1
    i += 1
valid_x = valid_x.transpose(0, 3, 1, 2)

In [12]:
def test_generator(df, datagen, batch_size = BATCHSIZE):
    n = df.shape[0]
    batch_index = 0
    while 1:
        current_index = batch_index * batch_size
        if n >= current_index + batch_size:
            current_batch_size = batch_size
            batch_index += 1    
        else:
            current_batch_size = n - current_index
            batch_index = 0        
        batch_df = df[current_index:current_index+current_batch_size]
        batch_x = np.zeros((batch_df.shape[0], ROWS, COLS, 3), dtype=K.floatx())
        i = 0
        for index,row in batch_df.iterrows():
            row = row.tolist()
            image_file = row[0]
            # typ_class = row[1]
            img = Image.open(image_file).resize((ROWS, COLS))
            img = img.convert('RGB')
            x = np.asarray(img, dtype=K.floatx())
            # x = datagen.random_transform(x)
            x = preprocess_input(x)
            batch_x[i] = x
            i += 1
        if batch_index%100 == 0: print(batch_index)
        # return (batch_x.transpose(0, 3, 1, 2))
        yield(batch_x.transpose(0, 3, 1, 2))

In [13]:
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto')        
model_checkpoint = ModelCheckpoint(filepath=CHECKPOINT_DIR+'weights.{epoch:03d}-{val_loss:.4f}.hdf5', monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=True, mode='auto')
# learningrate_schedule = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1, mode='auto', epsilon=0.001, cooldown=0, min_lr=0)
# tensorboard = TensorBoard(log_dir=LOG_DIR, histogram_freq=0, write_graph=False, write_images=True)

In [14]:
print "Model creation... "
nb_epoch = 5
# (img_rows, img_cols, color_type=1, num_classeses=None, dropout_keep_prob=0.2)
model = inception_v4_model(ROWS, COLS, channel, num_classes=num_class, dropout_keep_prob=0.2)
for layer in model.layers:
    layer.trainable = False
for layer in model.layers[-3:]:
    layer.trainable = True

# Start Fine-tuning
print "Fine tune part 1"
model.fit_generator(train_generator(train_datagen, train_df),
          nb_epoch=nb_epoch,
          samples_per_epoch=samples_per_epoch, #50000,
          verbose=1,
          validation_data=(valid_x, valid_y),
          callbacks=[early_stopping, model_checkpoint],
          )

Model creation... 
Fine tune part 1
Epoch 1/5
13888/13888 [==============================] - 1947s - loss: 0.7759 - acc: 0.5772 - val_loss: 0.5549 - val_acc: 0.7709
Epoch 2/5
13888/13888 [==============================] - 1957s - loss: 0.3368 - acc: 0.8085 - val_loss: 0.2701 - val_acc: 0.9037
Epoch 3/5
13888/13888 [==============================] - 1938s - loss: 0.1547 - acc: 0.8887 - val_loss: 0.1383 - val_acc: 0.9512
Epoch 4/5
13888/13888 [==============================] - 1937s - loss: 0.0847 - acc: 0.9104 - val_loss: 0.0620 - val_acc: 0.9787
Epoch 5/5
13888/13888 [==============================] - 1936s - loss: 0.0399 - acc: 0.9287 - val_loss: 0.0483 - val_acc: 0.9848


In [15]:
for layer in model.layers:
    layer.trainable = False
for layer in model.layers[-20:]:
    layer.trainable = True

In [16]:
# Start Fine-tuning
nb_epoch = 4
print "Fine tune part 1B"
model.fit_generator(train_generator(train_datagen, train_df),
          nb_epoch=nb_epoch,
          samples_per_epoch=samples_per_epoch, #50000,
          verbose=1,
          validation_data=(valid_x, valid_y),
          callbacks=[early_stopping, model_checkpoint],
          )

Fine tune part 1B
Epoch 1/4
13888/13888 [==============================] - 1938s - loss: 0.0320 - acc: 0.9324 - val_loss: 0.0242 - val_acc: 0.9933
Epoch 2/4
13888/13888 [==============================] - 1937s - loss: 0.0239 - acc: 0.9327 - val_loss: 0.0364 - val_acc: 0.9872
Epoch 3/4
13888/13888 [==============================] - 1938s - loss: 0.0161 - acc: 0.9333 - val_loss: 0.0449 - val_acc: 0.9829
Epoch 4/4
13888/13888 [==============================] - 1938s - loss: 0.0165 - acc: 0.9330 - val_loss: 0.0195 - val_acc: 0.9957


In [17]:
# Hack to solve issue on model loading : https://github.com/fchollet/keras/issues/4044
import glob
import h5py
model_files = sorted(glob.glob(CHECKPOINT_DIR + '*.hdf5'))
for model_file in model_files:
    print("Update '{}'".format(model_file))
    with h5py.File(model_file, 'a') as f:
        if 'optimizer_weights' in f.keys():
            del f['optimizer_weights']

Update 'log/checkpoint07/weights.000-0.0242.hdf5'
Update 'log/checkpoint07/weights.000-0.5549.hdf5'
Update 'log/checkpoint07/weights.001-0.2701.hdf5'
Update 'log/checkpoint07/weights.002-0.1383.hdf5'
Update 'log/checkpoint07/weights.003-0.0195.hdf5'
Update 'log/checkpoint07/weights.003-0.0620.hdf5'
Update 'log/checkpoint07/weights.004-0.0483.hdf5'


### Bag the predictions from a few epochs

In [28]:
import glob
import time
from IPython.lib.display import FileLink
timestr = time.strftime("%Y%m%d")

files = glob.glob(CHECKPOINT_DIR+'*')
# there is apparently overfitting on the later epochs so exclude the epochs where we unfroze the top layers
val_losses = [float(f.split('-')[-1][:-5]) for f in files]
min_id = np.array(val_losses).argsort()[:bags].tolist()

[array([[  8.36955130e-01,   1.99344680e-02,   1.43110365e-01],
        [  2.66181596e-05,   9.99951959e-01,   2.14751581e-05],
        [  9.94375825e-01,   4.60151955e-03,   1.02260814e-03],
        ..., 
        [  1.07146610e-04,   9.24942493e-01,   7.49503002e-02],
        [  2.20124421e-05,   2.11369749e-02,   9.78840947e-01],
        [  5.43637289e-05,   9.99937415e-01,   8.18345870e-06]], dtype=float32)]

In [36]:
# Loop the lowest val losses and get a prediction for each
test_preds_ls = []
model = inception_v4_model(ROWS, COLS, channel, num_classes=num_class, dropout_keep_prob=0.2)
for index in min_id:
    print('Loading model from checkpoints file ' + files[index])
    model.load_weights(files[index])
    test_model_name = files[index].split('/')[-2][-1:]+'_'+files[index].split('/')[-1]
    test_preds_ls.append(model.predict_generator(test_generator(test_df, train_datagen), 
                                         val_samples = test_df.shape[0])) 
    gc.collect()

Loading model from checkpoints file log/checkpoint07/weights.003-0.0195.hdf5
0
Loading model from checkpoints file log/checkpoint07/weights.000-0.0242.hdf5
0
Loading model from checkpoints file log/checkpoint07/weights.004-0.0483.hdf5
0
Loading model from checkpoints file log/checkpoint07/weights.003-0.0620.hdf5
0
Loading model from checkpoints file log/checkpoint07/weights.002-0.1383.hdf5
0


In [37]:
test_preds = sum(test_preds_ls)/len(test_preds_ls)

In [38]:
test_sub = pd.DataFrame(test_preds, columns=CERV_CLASSES)
test_sub['image_name'] = test_df['img'].str.split('/').apply(lambda x: x[-1])
test_sub = test_sub[['image_name'] + CERV_CLASSES ]
test_sub.head(3)

,image_name,Type_1,Type_2,Type_3
0,400.jpg,0.727681,0.062392,0.209927
1,430.jpg,0.010558,0.988899,0.000543
2,303.jpg,0.885383,0.060143,0.054474


In [39]:
if full:
    subm_name = '../sub/sub_dara_full_resnet_raw_5xbag_' + timestr + '.csv' #'.csv.gz'
else:
    subm_name = '../sub/sub_dara_part_resnet_raw_5xbag_' + timestr + '.csv' #'.csv.gz'
    
test_sub.to_csv(subm_name, index=False)#, compression='gzip')
FileLink(subm_name)

/home/ubuntu/cervical/sub/sub_dara_part_resnet_raw_5xbag_20170521.csv